In [56]:
using JuMP
using NamedArrays

<strong> Questions for this project:   </strong>
1. How to choose a team's strategy  
a. Knowning opponent's strategy  
b. Nash equilibrium: optimize for both 2 teams  


2. Players selection: how to buy some qualified players with a small budget   
a. Establish teams from scratch  
b. Pre-season transfer  

# Data and support functions

## Demo data

In [43]:
# one is enouxgh!! No of games - 1's score = 2's score
elo = [2838, 2822, 2817, 2772, 2771, 2761, 2750, 2747, 2745, 2741]
price = [1000, 900, 850, 700, 690, 650, 600, 580, 570, 560] * 0.001 

team_host = [1 3 5 6]
team_guest = [2 4 7 8]

plan_guest = [2 1 4 3] # fixed plan

N_PLAYERS = 4
N_CHOSEN = 4
N_POOL = 10
;

## Support functions

### Score function

In [17]:
function get_simple_score(player, opponent, order, elo)
    # first move => +0.75
    x = elo[player] - elo[opponent] + 0.75 * (-1)^(order + 1)
    if x > 0 
        return 1
    elseif x < 0
        return 0
    else 
        return 0.5
    end  
end
;

In [85]:
# the positional advantage and the ratings difference
# higher elo, first move better
# order: 1 or 0 with 1 being first
function get_score_chess_game(player, opponent, order, elo)
    x = elo[player] - elo[opponent] + 35*(-1)^(order + 1)
    return 1/(1 + 10^(-x/400)) 
end
;

### Match score function

In [98]:
function get_match_score(team_player, team_opponent, order, elo)
    sum(get_score_chess_game(team_player[i], team_opponent[i], 
            (i + order + 1) % 2, elo) for i=1:length(team_player))
end
;

# Choosing team's strategy

## Case 1: Fixed opponent's strategy

### Model 1: Simple score function

In [18]:
simpleModel = Model()

@variable(simpleModel, x[1:N_PLAYERS, 1:N_PLAYERS], Bin)

@constraint(simpleModel, supply[k in 1:N_PLAYERS], 
                sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(simpleModel, demand[k in 1:N_PLAYERS], 
                sum(x[i, k] for i=1:N_PLAYERS) == 1)

@objective(simpleModel, Max, sum(x[i, j] * get_simple_score(
            team_host[j], team_guest[plan_guest[i]], i % 2, elo) 
            for i=1:N_PLAYERS, j=1:N_PLAYERS))
                                    
solve(simpleModel)
                                    
xopt = getvalue(x)
println("Strategy: ")
for i in 1:N_PLAYERS
    for j in 1:N_PLAYERS
        if xopt[i, j] != 0
            println(team_guest[i], " - ", team_host[j])
        end
    end
end
println("Score: ", getobjectivevalue(simpleModel))

Strategy: 
2 - 3
4 - 1
7 - 6
8 - 5
Score: 4.0


### Model 2: Probabilistic score function

In [87]:
probModel = Model()

@variable(probModel, x[1:N_PLAYERS, 1:N_PLAYERS], Bin)

@constraint(probModel, supply[k in 1:N_PLAYERS], 
                sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(probModel, demand[k in 1:N_PLAYERS], 
                sum(x[i, k] for i=1:N_PLAYERS) == 1)

@objective(probModel, Max, sum(x[i, j] * get_score_chess_game(team_host[j], 
                            team_guest[plan_guest[i]], i % 2, elo) 
                            for i=1:N_PLAYERS, j=1:N_PLAYERS))
                                    
solve(probModel)
                                    
xopt = getvalue(x)
println("Strategy: ", )
for i in 1:N_PLAYERS
    for j in 1:N_PLAYERS
        if xopt[i, j] != 0
            println(team_guest[plan_guest[i]], " - ", team_host[j])
        end
    end
end
println("Score: ", getobjectivevalue(probModel))

Strategy: 
4 - 3
2 - 6
8 - 5
7 - 1
Score: 2.1381813962909337


## Nash equilibrium: no information about opponent's strategy

### Optimize our strategy

In [88]:
nashModel = Model()

@variable(nashModel, x[1:N_PLAYERS, 1:N_PLAYERS], Bin)
@variable(nashModel, p[1:N_PLAYERS])
@variable(nashModel, q[1:N_PLAYERS])
@variable(nashModel, r[1:N_PLAYERS, 1:N_PLAYERS] <= 0)

@constraint(nashModel, supply[k in 1:N_PLAYERS], 
                sum(x[k, j] for j=1:N_PLAYERS) == 1)
@constraint(nashModel, demand[k in 1:N_PLAYERS], 
                sum(x[i, k] for i=1:N_PLAYERS) == 1)
                        
@constraint(nashModel, mincons[j in 1:N_PLAYERS, k in 1:N_PLAYERS], 
    p[j] + q[k] + r[j, k] <= 
        sum(get_score_chess_game(team_host[i], team_guest[j], k, elo) 
                                * x[i, k] for i=1:N_PLAYERS))
                                    
@objective(nashModel, Max, sum(p) + sum(q) + sum(r))        

solve(nashModel)
                                    
println(getobjectivevalue(nashModel))
xopt = getvalue(x)
solution_x = NamedArray(Int[xopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.1368277781896134
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  0  1
2     │ 0  1  0  0
3     │ 1  0  0  0
4     │ 0  0  1  0
p: [0.472524,0.544223,0.575687,0.5799]
q: [0.0042012,-0.0298361,-0.00987175,0.0]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


###  Get opponent's best strategy

In [89]:
ym = Model()

@variable(ym, y[1:N_PLAYERS, 1:N_PLAYERS], Bin)

@constraint(ym, supply[k in 1:N_PLAYERS], sum(y[k, j] for j=1:N_PLAYERS) == 1)
@constraint(ym, demand[k in 1:N_PLAYERS], sum(y[i, k] for i=1:N_PLAYERS) == 1)

@expression(ym, score, sum(get_score_chess_game(team_host[i], 
                    team_guest[j], k, elo) * xopt[i, k] * y[j, k] 
                    for i=1:N_PLAYERS, j in 1:N_PLAYERS, k in 1:N_PLAYERS))
                                    
@objective(ym, Min, score)
            
solve(ym)
println(getobjectivevalue(ym))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[i, k] for i=1:N_PLAYERS, k=1:N_PLAYERS])
println(solution_y)

2.1368277781896134
4×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  1  0
2     │ 0  1  0  0
3     │ 0  0  0  1
4     │ 1  0  0  0


# Team Selection and Trade off

We don't know other's strategy, hence, we use the general solution (Nash)   
Oppenents select their teams from the rest players

## Model 1: Selecting from a pool

### Optimize our strategy

In [90]:
inf = -1000
sup = 1000

xm = Model()

@variable(xm, x[1:N_POOL, 1:N_CHOSEN], Bin)
@variable(xm, p[1:N_POOL] <= 0) #dual corresponding to <= 1
@variable(xm, q[1:N_CHOSEN])
@variable(xm, r[1:N_POOL, 1:N_CHOSEN] <= 0)
@variable(xm, t[1:N_POOL] <= 0)

@constraint(xm, supply[i in 1:N_POOL], sum(x[i, k] for k=1:N_CHOSEN) <= 1)
@constraint(xm, demand[k in 1:N_CHOSEN], sum(x[i, k] for i=1:N_POOL) == 1)
                        
@constraint(xm, icons[j in 1:N_POOL], t[j] >= 
                            (1 - sum(x[j, k] for k=1:N_CHOSEN)) * inf)
@constraint(xm, ucons[j in 1:N_POOL], t[j] <= 
                            p[j] + sup*sum(x[j, k] for k=1:N_CHOSEN))
                        
@constraint(xm, mincons[j in 1:N_POOL, k in 1:N_CHOSEN], 
    p[j] + q[k] + r[j, k]  <= sum(get_score_chess_game(i, j, k, elo) 
                                                * x[i, k] for i=1:N_POOL))
                                                    
@objective(xm, Max, sum(t) + sum(q) + sum(r))        

solve(xm)
println(getobjectivevalue(xm))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:N_POOL, k=1:N_CHOSEN])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

2.3111300532279953
10×4 Named Array{Float64,2}
A ╲ B │   1    2    3    4
──────┼───────────────────
1     │ 0.0  1.0  0.0  0.0
2     │ 0.0  0.0  0.0  1.0
3     │ 0.0  0.0  1.0  0.0
4     │ 1.0  0.0  0.0  0.0
5     │ 0.0  0.0  0.0  0.0
6     │ 0.0  0.0  0.0  0.0
7     │ 0.0  0.0  0.0  0.0
8     │ 0.0  0.0  0.0  0.0
9     │ 0.0  0.0  0.0  0.0
10    │ 0.0  0.0  0.0  0.0
p: [-1000.0,-1000.0,-1000.0,-1000.0,-0.0365883,-0.0223096,-0.00682331,-0.0026261,0.0,0.0]
q: [0.588125,0.58251,0.649295,0.559657]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -5.89756e-5; 0.0 -5.05318e-5 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


###  Get opponent's best strategy

In [91]:
ym = Model()

@variable(ym, y[1:N_POOL, 1:N_CHOSEN], Bin)

@constraint(ym, supply[j in 1:N_POOL], sum(y[j, k] for k=1:N_CHOSEN) 
            <= 1 - sum(xopt[j, k] for k=1:N_CHOSEN))
@constraint(ym, demand[k in 1:N_CHOSEN], 
                        sum(y[i, k] for i=1:N_POOL) == 1)
                        
@objective(ym, Min, sum(get_score_chess_game(i, j, k, elo) 
                        * xopt[i, k] * y[j, k] 
                        for i=1:N_POOL, j=1:N_POOL, k=1:N_CHOSEN))        
solve(ym)
                                            
println(getobjectivevalue(ym))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[i, k] for i=1:N_POOL, k=1:N_CHOSEN])
println(solution_y)

2.311130053223995
10×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  0  0
2     │ 0  0  0  0
3     │ 0  0  0  0
4     │ 0  0  0  0
5     │ 0  0  0  1
6     │ 0  1  0  0
7     │ 1  0  0  0
8     │ 0  0  1  0
9     │ 0  0  0  0
10    │ 0  0  0  0


## Model 2 - Selecting with a small budget
Problem:   
- opponent doesn't care about our budget => different objective 
- assume that opponent is very rich   

### Optimize our strategy

In [92]:
inf = -1000
sup = 1000
lambda = 0.5

xm = Model()

@variable(xm, x[1:N_POOL, 1:N_CHOSEN], Bin)
@variable(xm, p[1:N_POOL] <= 0) #dual corresponding to <= 1
@variable(xm, q[1:N_CHOSEN])
@variable(xm, r[1:N_POOL, 1:N_CHOSEN] <= 0)
@variable(xm, t[1:N_POOL] <= 0)

@constraint(xm, supply[i in 1:N_POOL], sum(x[i, k] for k=1:N_CHOSEN) <= 1)
@constraint(xm, demand[k in 1:N_CHOSEN], sum(x[i, k] for i=1:N_POOL) == 1)
                        
@constraint(xm, icons[j in 1:N_POOL], t[j] >= 
                            (1 - sum(x[j, k] for k=1:N_CHOSEN)) * inf)
@constraint(xm, ucons[j in 1:N_POOL], t[j] <= 
                            p[j] + sup * sum(x[j, k] for k=1:N_CHOSEN))
                        
@constraint(xm, mincons[j in 1:N_POOL, k in 1:N_CHOSEN], 
    p[j] + q[k] + r[j, k]  <= sum((get_score_chess_game(i, j, k, elo) 
                         - lambda * price[i]) * x[i, k] for i=1:N_POOL))
                                                    
@objective(xm, Max, sum(t) + sum(q) + sum(r))        

solve(xm)
println(getobjectivevalue(xm))
xopt = getvalue(x)
solution_x = NamedArray([xopt[i, k] for i=1:N_POOL, k=1:N_CHOSEN])
println(solution_x)
println("p: ", getvalue(p))
println("q: ", getvalue(q))
println("r: ", getvalue(r))

0.5882920809665445
10×4 Named Array{Float64,2}
A ╲ B │   1    2    3    4
──────┼───────────────────
1     │ 0.0  0.0  0.0  1.0
2     │ 0.0  1.0  0.0  0.0
3     │ 1.0  0.0  0.0  0.0
4     │ 0.0  0.0  0.0  0.0
5     │ 0.0  0.0  1.0  0.0
6     │ 0.0  0.0  0.0  0.0
7     │ 0.0  0.0  0.0  0.0
8     │ 0.0  0.0  0.0  0.0
9     │ 0.0  0.0  0.0  0.0
10    │ 0.0  0.0  0.0  0.0
p: [-1000.0,-1000.0,-1000.0,-0.0351106,-1000.0,-0.019455,-0.00395577,0.0,0.0,0.0]
q: [0.221669,0.106684,0.238856,0.0796052]
r: [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]


###  Get opponent's best strategy

In [93]:
ym = Model()

@variable(ym, y[1:N_POOL, 1:N_CHOSEN], Bin)

@constraint(ym, supply[j in 1:N_POOL], sum(y[j, k] for k=1:N_CHOSEN) 
            <= 1 - sum(xopt[j, k] for k=1:N_CHOSEN))
@constraint(ym, demand[k in 1:N_CHOSEN], 
                        sum(y[i, k] for i=1:N_POOL) == 1)
                                
# prices of x players are constant for y 
@expression(ym, score, sum(get_score_chess_game(i, j, k, elo) * xopt[i, k] * y[j, k] 
                        for i=1:N_POOL, j=1:N_POOL, k=1:N_CHOSEN))
                                            
@objective(ym, Min, sum((get_score_chess_game(i, j, k, elo) 
                - lambda * price[i]) * xopt[i, k] * y[j, k] 
                        for i=1:N_POOL, j=1:N_POOL, k=1:N_CHOSEN))        
solve(ym)
                                            
println(getobjectivevalue(ym))
println("Price: ", getvalue(score))
yopt = getvalue(y)
solution_y = NamedArray(Int[yopt[i, k] for i=1:N_POOL, k=1:N_CHOSEN])
println(solution_y)

0.5882920809665444
Price: 2.3082920809665444
10×4 Named Array{Int64,2}
A ╲ B │ 1  2  3  4
──────┼───────────
1     │ 0  0  0  0
2     │ 0  0  0  0
3     │ 0  0  0  0
4     │ 0  1  0  0
5     │ 0  0  0  0
6     │ 0  0  0  1
7     │ 0  0  1  0
8     │ 1  0  0  0
9     │ 0  0  0  0
10    │ 0  0  0  0


In [103]:
score = get_match_score([3, 2, 5, 1], [8, 4, 7, 6], 1, elo)

2.3082920809665444